[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/nils-holmberg/socs-qmd/blob/main/jnb/lecture3_nlp3.ipynb)

# chatgpt test (topic modeling)

prompt:

In [ ]:
# paste chatgpt code here..
















In [ ]:
import pandas as pd
import numpy as np
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Download stopwords if not already available
nltk.download('stopwords')
nltk.download('punkt')

# Load your Excel file
excel_file_path = 'your_excel_file.xlsx'
data = pd.read_excel(excel_file_path)

# Assuming your text data is in a column named 'text_column'
text_data = data['text_column']

# Tokenization and preprocessing
stop_words = set(stopwords.words('english'))  # English stopwords
stop_words = stopwords.words('english')
vectorizer = CountVectorizer(max_df=0.95, min_df=2, stop_words=stop_words, token_pattern=r'\b\w+\b')
tf = vectorizer.fit_transform(text_data)

# Apply LDA topic modeling
num_topics = 4  # You can adjust the number of topics
lda = LatentDirichletAllocation(n_components=num_topics, random_state=42)
lda.fit(tf)

# Print topics and associated words
feature_names = vectorizer.get_feature_names_out()
num_top_words = 10  # You can adjust the number of top words per topic

for topic_idx, topic in enumerate(lda.components_):
    print(f"Topic {topic_idx + 1}:")
    top_words_idx = topic.argsort()[:-num_top_words - 1:-1]
    top_words = [feature_names[i] for i in top_words_idx]
    print(" ".join(top_words))


# text classification (naive bayes)

In [ ]:
!gdown https://drive.google.com/uc?id=1EMzJxxoBaN_NbvF7xhoc09K82vQ6H_LX

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
fp = "content.xlsx"
df = pd.read_excel(fp, header=0, sheet_name='reviews')
df.head()

In [ ]:
import nltk
from nltk.corpus import stopwords

# Download the NLTK stopwords if not already downloaded
nltk.download('stopwords')


## define features and outcomes

## split training and test data

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

# Step 1: Assuming your Excel columns are named 'text' and 'sentiment'
X = df['text']  # Text data
y = df['sentiment']  # Binary sentiment labels

# Step 2: Perform TF-IDF vectorization
tfidf_vectorizer = TfidfVectorizer(max_features=1000)  # You can adjust the number of features as needed
X_tfidf = tfidf_vectorizer.fit_transform(X)

# Step 3: Split the dataset into training (750 rows) and testing (250 rows)
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.25, random_state=42)

# Now, X_train and y_train contain the training data and labels (750 rows),
# and X_test and y_test contain the testing data and labels (250 rows).


## train naive bayes classifier

## validate model performance

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Step 4: Train a Multinomial Naive Bayes classifier
clf = MultinomialNB()
clf.fit(X_train, y_train)

# Step 5: Make predictions on the test data
y_pred = clf.predict(X_test)

# Step 6: Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-score: {f1:.2f}")

# Step 7: Create and plot a confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(4, 2))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", cbar=False)
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix")
plt.show()


## save classification model

## make inference on new samples

In [ ]:
import joblib

# Save the trained model and TF-IDF vectorizer to files
model_filename = 'sentiment_model.pkl'
vectorizer_filename = 'tfidf_vectorizer.pkl'

joblib.dump(clf, model_filename)
joblib.dump(tfidf_vectorizer, vectorizer_filename)


In [ ]:
# To load the model and vectorizer for inference:
loaded_model = joblib.load(model_filename)
loaded_vectorizer = joblib.load(vectorizer_filename)

# Example: Using the loaded model and vectorizer to predict sentiment for a new sample string
new_sample = "This phone is amazing"
# Vectorize the new sample using the loaded vectorizer
new_sample_tfidf = loaded_vectorizer.transform([new_sample])
# Use the loaded model for prediction
prediction = loaded_model.predict(new_sample_tfidf)

# Convert the prediction to a human-readable label if needed
sentiment_label = "Positive" if prediction == 1 else "Negative"

print(f"Predicted sentiment: {sentiment_label}")


# spacy language models

In [ ]:
# medium size language model
!python -m spacy download en_core_web_md


In [ ]:
import spacy

# Load the spaCy model
nlp = spacy.load("en_core_web_sm")

# Define random words
word = "cat"

# Process the words to get their word vectors
token = nlp(word)

# Check if the token has a vector
if token.has_vector:
    # Print the word vector as a NumPy array
    print(f"Vector for '{word}':")
    print(token.vector)
else:
    print(f"No vector available for '{word}'.")


In [ ]:
print(len(token.vector))

In [ ]:
# Define two random words
word1 = "cat"
word2 = "tiger"

# Process the words to get their word vectors
token1 = nlp(word1)
token2 = nlp(word2)

# Check if the tokens are valid words (have word vectors)
if token1.has_vector and token2.has_vector:
    # Compute the cosine similarity between the word vectors
    similarity = token1.similarity(token2)
    print(f"Similarity between '{word1}' and '{word2}': {similarity}")
else:
    print("One or both of the words do not have word vectors available in the spaCy model.")
